# Data Collection

### Import Libraries For Data Collection

In [ ]:
from tapsdk import TapSDK, TapInputMode
from tapsdk.models import AirGestures
import pandas as pd
import numpy as np
import keyboard
import time
import os 
import glob

tap_instance = []
tap_identifiers = []
global x_array 
global y_array 

modes = ["text", "controller", "controller_text", "raw"]

### Functions

In [ ]:
class Functionality:
    current_x = None
    current_y = None
    
    # For connection notification
    def on_connect(self, identifier, name, fw):
        print("Tap has connected")
        print(identifier + " Tap: " + str(name), " FW Version: ", fw)
        if identifier not in tap_identifiers:
            tap_identifiers.append(identifier)
        print("Connected taps:")
        for identifier in tap_identifiers:
            print(identifier)

    # For disconnection notification
    def on_disconnect(self, identifier):
        print("Tap has disconnected")
        if identifier in tap_identifiers:
            tap_identifiers.remove(identifier)
        for identifier in tap_identifiers:
            print(identifier)

    # For mouse data
    def on_mouse_event(self, identifier, dx: int, dy: int, isMouse):
        if isMouse:
            self.current_x = dx 
            self.current_y = dy
            # print(str(dx), str(dy))
        else:
            pass
            # print("Air: ", str(dx), str(dy))

    #For adding columns on top 
    def add_top_column(self, df: pd.DataFrame, top_col: str, inplace=False):
        if not inplace:
            df = df.copy()
        
        df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
        return df

### Data Collection

#### Non-Automated Collection

In [ ]:
def main(): 
    count = 0 
    start = input(f"Type 'ST' to start running")
    
    while start == "ST":
        global tap_instance
        tap_instance = TapSDK()
        fun = Functionality()
        tap_instance.run()

        # tap_instance.register_connection_events(on_connect)
        # tap_instance.register_disconnection_events(on_disconnect)
        tap_instance.register_mouse_events(fun.on_mouse_event)
        tap_instance.set_input_mode(TapInputMode(modes[2]))   

        trial = True 
        while trial:
            if(keyboard.is_pressed("q")):
                trial = False
                print("You have chosen to end this trial.")
            else: 
                if str(fun.current_x) != "None":
                    # print(str(fun.current_x) + " "+ str(fun.current_y) + " " + "got it")
                    with open(f"Data\\Random_+1\\random_{count}.txt", "a") as my_file:
                        my_file.write(str(fun.current_x) + " " + str(fun.current_y) +"\n")
                pass

        count += 1
        if  count <= 100:  
            start = input("Type 'ST' to keep running")
        else: 
            start = "Finish"
            print(start)

if __name__ == "__main__":
    main() 

#### Data Cleaning and Dumping

In [ ]:
# Remove repetitive lines from both CW and CCW text files 
path1 = r"C:\Users\MoRevolution\Desktop\College\Classes\2022-2023\J-term\CSC-380\Tap Strap Functionality\Data\Random"
listing = glob.glob(path1 + '\\*.txt')
count = 0 
for file in listing:
    lines_seen = set() # holds lines already seen
    outfile = open(f"Data\\Cleaned_Random\\random_{count}.txt", "a")
    for line in open(file, "r"):
        if line not in lines_seen: # not a duplicate
            outfile.write(line)
            lines_seen.add(line)
    outfile.close()
    count += 1 

In [ ]:
#Add data from text files to csv
trial_df = pd.DataFrame()
count = 0 
path1 = r"C:\Users\MoRevolution\Desktop\College\Classes\2022-2023\J-term\CSC-380\Tap Strap Functionality\Data\Cleaned_Random"
listing = glob.glob(path1 + '\\*.txt')
for file in listing:
    x_array = np.array([])
    y_array = np.array([])
    # con = input("Press Y")
    with open(file, 'r') as my_file: 
        for line in my_file:
            line = line.split(" ")
            x_array = np.append((x_array), [int(line[0])])
            y_array = np.append((y_array), [int(line[1])])
    trial_column = pd.DataFrame()
    trial_column[f"x_{count}"] = x_array.tolist()
    trial_column[f"y_{count}"] = y_array.tolist()
    trial_df= pd.concat([trial_df, trial_column],axis=1)
    count += 1 
print("Finished all!!")
trial_df.to_csv(input("Type file name"), index=False)

#### Experimental Automated Collection

In [ ]:
def main(): 
    trial_df = pd.DataFrame()
    count = 0
    start = input(f"Type 'St' to start running")

    while start == "St":
        trial = input(f"Type 'ST' to start running this trial")
        x_array = np.array([])
        y_array = np.array([])
        while trial == "ST":
            print("We are on trial" + str(count))

            # instantiate a Tap object and start registering events
            global tap_instance
            tap_instance = TapSDK()
            fun = Functionality()
            tap_instance.run()
            tap_instance.register_mouse_events(fun.on_mouse_event)

            #use 1 or 3 for controller or raw data mode respectively in list modes
            tap_instance.set_input_mode(TapInputMode(modes[1]))   

            # Command to end the recording of one trail within a case
            if(keyboard.is_pressed("q")):
                trail = "END"
                print("You have chosen to end this trial.")
            else:          
                #append each position change value to the array 
                x_array = np.append(x_array,[fun.current_x])
                y_array = np.append(x_array,[fun.current_y])   
                print(str(fun.current_x), str(fun.current_y)) 
                pass

            # Command to end the recording of trials within this database
        if (keyboard.is_pressed('p')): 
            start = "End"
            print("You have chosen to end this iteration of trials. Saving data...")
        else:  
            #Add this trial to the trial_df dataframe
            trial_column = pd.DataFrame([[x_array, y_array]], columns=['x_'+ str(count), 'y_'+ str(count)])
            # trial_column = add_top_column(trial_column, "Trial_" + str(count))
            trial_df= pd.concat([trial_df, trial_column])

            print(f"Trial {count} just ended, starting next trial. ")
            count += count
            print(f"Trial {count} is next")
            pass

    trial_df.to_csv(input("Type in the name for this csv file"), index=False)

if __name__ == "__main__":
    main()

In [ ]:
def main():
    trial_df = pd.DataFrame()
    count = 0
    start = input(f"Type 'St' to start running")

    while start == "St":
        trial = input(f"Type 'ST' to start running this trial")
        x_array = np.array([])
        y_array = np.array([])
        while trial == "ST":
            print("We are on trial" + str(count))

            # instantiate a Tap object and start registering events
            global tap_instance
            tap_instance = TapSDK()
            tap_instance.run()
            tap_instance.register_connection_events(on_connect)
            tap_instance.register_disconnection_events(on_disconnect)
            tap_instance.register_mouse_events(on_mouse_event)
            
            #append each position change value to the array 
            x_array = np.append(x_array,[dx])
            y_array = np.append(x_array,[dy])   
            print(str(dx), str(dy)) 

            #use 1 or 3 for controller or raw data mode respectively in list modes
            tap_instance.set_input_mode(TapInputMode(modes[1])) 

            # Command to end the recording of one trail within a case
            if(keyboard.is_pressed("q")):
                trail = "END"
                print("You have chosen to end this trial.")
            else: 
                pass
        # Command to end the recording of trials within this database
        if (keyboard.is_pressed('p')): 
            start = "End"
            print("You have chosen to end this iteration of trials. Saving data...")
        else:  
            #Add this trial to the trial_df dataframe
            trial_column = pd.DataFrame([[x_array, y_array]], columns=['x_'+ str(count), 'y_'+ str(count)])
            # trial_column = add_top_column(trial_column, "Trial_" + str(count))
            trial_df= pd.concat([trial_df, trial_column])

            print(f"Trial {count} just ended, starting next trial. ")
            count += count
            print(f"Trial {count} is next")
            pass

    trial_df.to_csv(input("Type in the name for this csv file"), index=False)

if __name__ == "__main__":
    main()

# Model Training and Evaluation

### Data Pre-Processing

#### Import Libraries

In [ ]:
import pandas as pd

#### Relative to Non-Relative

In [ ]:
"""
Data given by tapstrap 2 is in the form of relative coordinates, that should be changed to 
non-relative coordinates.
"""
#Dataframe for non-relative coordinates(x,y)
trial_df = pd.DataFrame()

#import dataframe with relative coordinates (x,y)
data = pd.read_csv("Data\\Cleaned_Circle_+1.csv")

for count in range(100):
    #starting point for the drawing of each circle
    origin_x = 0 
    origin_y = 0 
    #array containing calculated coordinates
    new_x_array = np.array([origin_x])
    new_y_array = np.array([origin_y])

    data_temp = data[[f"x_{count}" , f"y_{count}"]]
    data_temp = data_temp.dropna(how='any',axis=0) 
    # data = data.drop_duplicates(keep='first')
    
    for x,y in zip(data_temp[f'x_{count}'].tolist(), data_temp[f'y_{count}'].tolist()): 
        origin_x += x 
        origin_y += y 
        new_x_array = np.append(new_x_array, [origin_x])
        new_y_array = np.append(new_y_array, [origin_y])

    new_df = pd.DataFrame()
    new_df[f"x_{count}"] = new_x_array.tolist()
    new_df[f"y_{count}"] = new_y_array.tolist()
    trial_df = pd.concat([trial_df, new_df], axis= 1)
trial_df.to_csv(input("Type in file name"), index=False)

#### Data Normalization

In [ ]:

"""Data was normalized using z-score normalization or standardization.
"""
from sklearn.preprocessing import StandardScaler

data_array = [] # array for non-normalized data
norm_data_array = [] #array for normalized data

#import non-relative coordinate data
data_array.append(pd.read_csv("Data\\Non-Relative_Circle_+1.csv"))
data_array.append(pd.read_csv("Data\\Non-Relative_Circle_-1.csv"))
data_array.append(pd.read_csv("Data\\Non-Relative_Random.csv"))

#iterate through each dataframe in data_array
for data in data_array: 
    data_new = pd.DataFrame()
    for count in range(100):
        #normalize a subset of data
        data_temp = data[[f'x_{count}', f'y_{count}']].dropna(how='any',axis=0) 
        data_temp = data_temp.to_numpy()

        #create standardizing object and normalize aformentioned subset
        scaler = StandardScaler()
        scaler.fit(data_temp)
        data_temp = scaler.transform(data_temp)
        data_temp = pd.DataFrame(data_temp, columns = [f'x_{count}', f'y_{count}'])
        data_new = pd.concat([data_new, data_temp], axis=1)
    
    norm_data_array.append(data_new)

In [ ]:
cw = norm_data_array[0]
ccw = norm_data_array[1]
rand_= norm_data_array[2]

temp = cw.copy()
temp_arr = temp.iloc[:,0:2].to_numpy()

# # df.drop(df.loc[df.time< df.time.shift()].index, inplace=True)
# df = df.drop(df.columns[:2], axis=1)
temp = temp.drop(temp.columns[:2], axis=1)
temp


In [ ]:
import numpy as np
ar1= cw.iloc[:, 0:1].dropna(how='any', axis=0).to_numpy()
ar2 =cw.iloc[:, 1:2].dropna(how='any', axis=0).to_numpy()
out =  np.full(ar1.shape, 1)
temporary =  np.hstack((ar1, ar2, out))
temporary.shape
data = pd.DataFrame(temporary, columns = [f'x_{0}', f'y_{0}',f'out_{0}'])
data

In [ ]:
# label = np.array([0])
# for x,y in zip(cw[f'x_{0}'].dropna(how='any',axis=0).to_list(), cw[f'y_{0}'].dropna(how='any',axis=0).tolist()):
    
x = cw[f'x_{0}'].dropna(how='any',axis=0).to_numpy()
y = cw[f'y_{0}'].dropna(how='any',axis=0).to_numpy()
out =  np.full(y.shape, 1, dtype=int)
combined_array = np.vstack((x, y, out)).T
temp = pd.DataFrame(combined_array, columns = ['x', 'y','out'])
temp

#### Experimental Combine CW and CCW

In [ ]:
""" This is to randomize the combination of the CW and CCW data.
"""
import random
import numpy as np

mixedFinalData = pd.DataFrame()
count = 0

cw = norm_data_array[0]
ccw = norm_data_array[1]
rand_= norm_data_array[2]

dataframes = [cw, ccw, rand_]

while len(cw.columns)/2 + len(ccw.columns)/2 + len(rand_.columns)/2 > 0:
  random_dataframe = random.choice(dataframes)
  if len(random_dataframe) > 0:
    
    ar1 = random_dataframe.iloc[:, 0:1].dropna(how='any', axis=0).to_numpy()
    ar2 = random_dataframe.iloc[:, 1:2].dropna(how='any', axis=0).to_numpy()

    if (random_dataframe.equals(cw) or random_dataframe.equals(ccw)):
      num = 1
    else:
      num = 0
    
    out =  np.full(ar1.shape, num , dtype=int)
      
    temp_array = np.hstack((ar1, ar2, out))
    temp_frame = pd.DataFrame(temp_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])

    random_dataframe = random_dataframe.drop(random_dataframe.columns[:2], axis=1)

    mixedFinalData = pd.concat([mixedFinalData, temp_frame], axis=1)
    # increase count if a dataframe was concatenated
    count += 1 

mixedFinalData.to_csv(input("Type in a name"), index=False)

In [ ]:
""" 
Plot the non-relative (x,y) and save it
"""
import matplotlib.pyplot as plt

data = pd.read_csv("Data\\Non-Relative_Random.csv")

plt.switch_backend('agg')

for count in range(100): 
    data_temp = data[[f'x_{count}', f'y_{count}']].dropna(how='any',axis=0)  #subset of dataframe

    # make data
    x,y = data_temp[f'x_{count}'].to_numpy(), data_temp[f'y_{count}'].to_numpy() # from dataframe to arrays containing x and y coordinates

    # naming the x and y axis
    plt.xlabel('x - axis')
    plt.ylabel('y - axis')

    # # plotting a line plot after changing it's width and height
    # f = plt.figure()
    # f.set_figwidth(5)
    # f.set_figheight(5)
    
    plt.plot(x, y, linewidth=5)
    # save the plot to a file
    filename = f"Data\Image Data\Random\_plot_{count}.png"
    plt.savefig(filename, dpi = 100, bbox_inches='tight')

    # clear the plot for the next iteration
    plt.clf()

##### Trial and Error

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# create a pandas dataframe with sample data
df = pd.DataFrame({
    'x': [1, 2, 3, 4],
    'y1': [2, 4, 6, 8],
    'y2': [3, 6, 9, 12],
    'y3': [4, 8, 12, 16]
})

# set Matplotlib backend to save plots to files
plt.switch_backend('agg')

# loop through each column in the dataframe
for column in df.columns[1:]:
    plt.plot(df['x'], df[column], 'ro', linewidth=5)
    plt.axis([0, 6, 0, 20])

    # save the plot to a file
    filename = "plot_{}.png".format(column)
    plt.savefig(filename)

    # clear the plot for the next iteration
    plt.clf()

In [ ]:
#save each plot 
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# cw = norm_data_array[0]
# ccw = norm_data_array[1]
# rand_= norm_data_array[2]

# dataframes = [cw, ccw, rand_]

# for dataframe in dataframes: 
    

plt.plot([0, 1, 2, 3, 4], [0, 3, 5, 9, 11])

plt.xlabel('Months')
plt.ylabel('Books Read')

# save the plot as a grayscale image
fig = plt.gcf()
fig.canvas.draw()
image = Image.frombytes("RGB", fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
gray_image = image.convert("L")

# resize the image to 28x28 pixels
resized_image = gray_image.resize((28, 28))

# save the resized image to a file
resized_image.save("resized_image.png")
# gray_image = np.array(Image.fromarray(gray_image).resize((28, 28)))

# # save the image to a file
# Image.fromarray(gray_image).save('plot_28x28_grayscale.png')




# fig1 = plt.gcf()
# fig1.savefig('ty.png', dpi=100, cmap='gray')
# plt.show()
# # plt.draw()



In [ ]:
""" This is to randomize the combination of the CW and CCW data.
"""

import random
import numpy as np

cw = norm_data_array[0]
ccw = norm_data_array[1]
rand_= norm_data_array[2]

mixedFinalData = pd.DataFrame()
cw_set = set()
cww_set= set()
rand_set = set()
count = 0
overall_count = [0, 100, 200]
oc_index = 0 

string_array = ['CW', 'CCW', 'random']


for times in range(300):
    random_string = random.choice(string_array)
    if (random_string == "CW"):
        if count not in cw_set:
            cw_set.add(count)
            prev_count = count 

            x = cw[f'x_{count}'].dropna(how='any',axis=0).to_numpy()
            y = cw[f'y_{count}'].dropna(how='any',axis=0).to_numpy()
            out =  np.full(y.shape, 1, dtype=int)
            combined_array = np.vstack((x, y, out)).T

            count = count + overall_count[oc_index]
            tempFrame = pd.DataFrame(combined_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])
            mixedFinalData = pd.concat([mixedFinalData, tempFrame], axis=1)
            count = prev_count
            
    elif(random_string == "CCW"):
        if count not in cww_set:
            cww_set.add(count)
            prev_count = count 

            x = ccw[f'x_{count}'].dropna(how='any',axis=0).to_numpy()
            y = ccw[f'y_{count}'].dropna(how='any',axis=0).to_numpy()
            out =  np.full(y.shape, 1, dtype=int)
            combined_array = np.vstack((x, y, out)).T

            count = count + overall_count[oc_index]
            tempFrame = pd.DataFrame(combined_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])
            mixedFinalData = pd.concat([mixedFinalData, tempFrame], axis=1)
            count = prev_count
    else: 
    
        if count not in rand_set:
            rand_set.add(count)
            prev_count = count 
            x = rand_[f'x_{count}'].dropna(how='any',axis=0).to_numpy()
            y = rand_[f'y_{count}'].dropna(how='any',axis=0).to_numpy()
            out =  np.full(y.shape, 0, dtype=int)
            combined_array = np.vstack((x, y, out)).T
            
            count = count + overall_count[oc_index]
            tempFrame = pd.DataFrame(combined_array, columns = [f'x_{count}', f'y_{count}',f'out_{count}'])
            mixedFinalData = pd.concat([mixedFinalData, tempFrame], axis=1)
            count = prev_count
    
    count += 1 
    if(count == 100): 
        oc_index += 1 
        count = 0
    else: 
        pass

mixedFinalData.to_csv(input("Type in a name"), index=False)

## SVM

### Take Input

In [ ]:
#Source: https://medium.com/analytics-vidhya/image-classification-using-machine-learning-support-vector-machine-svm-dc7a0ec92e01

import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt

Categories=['1','1','0']
dirs = ['-1', '+1', 'Random'] 
flat_data_arr=[] #input array
target_arr=[] #output array

dataPath = 'Data\Image Data'
#path which contains all the categories of images
for dir,i in zip(dirs, Categories):
    print(f'loading... category : {dir}')
    path= os.path.join(dataPath,dir)
    for img in os.listdir(path):
        img_array=imread(os.path.join(path,img))
        img_resized=resize(img_array,(150,150,3))
        flat_data_arr.append(img_resized.flatten())
        target_arr.append(Categories.index(i))
    print(f'loaded category:{dir} successfully')
    
flat_data=np.array(flat_data_arr)
target=np.array(target_arr)
df=pd.DataFrame(flat_data) #dataframe
df['Target']=target
x=df.iloc[:,:-1] #input data 
y=df.iloc[:,-1] #output data

### Model Construction

In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid)

### Model Training

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=77,stratify=y)
print('Splitted Successfully')
model.fit(x_train,y_train)
print('The Model is trained well with the given images')
# model.best_params_ contains the best parameters obtained from GridSearchCV

In [ ]:
from sklearn.metrics import accuracy_score

y_pred=model.predict(x_test)
print("The predicted Data is :")
print(y_pred)
print("The actual data is:")
print(np.array(y_test))
print(f"The model is {accuracy_score(y_pred,y_test)*100}% accurate")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = model.predict(x_test)
print(classification_report(y_test, predictions))

In [ ]:
# print(model.best_params_)
print(model.best_estimator_)

In [ ]:

filename = 'finalized_model_V1_NODIR.sav'
pickle.dump(model, open(filename, 'wb'))

### Model Evaluation

In [ ]:
import pickle
loaded_model = pickle.load(open('Models\\finalized_model_V1_NODIR.sav', 'rb'))
# print(model.best_params_)
print(loaded_model.best_estimator_)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
predictions = loaded_model.predict(x_test)
print(classification_report(y_test, predictions))

#### Functions

In [52]:
from tapsdk import TapSDK, TapInputMode
from tapsdk.models import AirGestures
import pandas as pd
import numpy as np
import keyboard
import time
import os 
import glob
from skimage.transform import resize
from skimage.io import imread

tap_instance = []
tap_identifiers = []
global x_array 
global y_array 

modes = ["text", "controller", "controller_text", "raw"]
class Functionality:
    current_x = None
    current_y = None
    
    # For connection notification
    def on_connect(self, identifier, name, fw):
        print("Tap has connected")
        print(identifier + " Tap: " + str(name), " FW Version: ", fw)
        if identifier not in tap_identifiers:
            tap_identifiers.append(identifier)
        print("Connected taps:")
        for identifier in tap_identifiers:
            print(identifier)

    # For disconnection notification
    def on_disconnect(self, identifier):
        print("Tap has disconnected")
        if identifier in tap_identifiers:
            tap_identifiers.remove(identifier)
        for identifier in tap_identifiers:
            print(identifier)

    # For mouse data
    def on_mouse_event(self, identifier, dx: int, dy: int, isMouse):
        if isMouse:
            self.current_x = dx 
            self.current_y = dy
            # print(str(dx), str(dy))
        else:
            pass
            # print("Air: ", str(dx), str(dy))

    #For adding columns on top 
    def add_top_column(self, df: pd.DataFrame, top_col: str, inplace=False):
        if not inplace:
            df = df.copy()
        
        df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
        return df
    # For changing relative coordinates to non-relative coordinates
    def reltoNonrel(self, x_arr, y_arr): 

        #starting point for the drawing of trajectory
        origin_x = 0 
        origin_y = 0 
        
        #array containing calculated coordinates
        new_x_array = np.array([origin_x])
        new_y_array = np.array([origin_y])

            # data_temp = data[[f"x_{count}" , f"y_{count}"]]
            # data_temp = data_temp.dropna(how='any',axis=0) 
            # # data = data.drop_duplicates(keep='first')
            
        for x,y in zip(x_arr, y_arr): 
            origin_x += x 
            origin_y += y 
            new_x_array = np.append(new_x_array, [origin_x])
            new_y_array = np.append(new_y_array, [origin_y])

        return x_arr, y_arr 
    # For drawing a plot of an x and y array and retuning the plotted image
    def plotfig(self, x_arr, y_arr): 
        import matplotlib.pyplot as plt
        import numpy as np
        
        plt.switch_backend('agg')

        # naming the x and y axis
        plt.xlabel('x - axis')
        plt.ylabel('y - axis')

        plt.plot(x, y, linewidth=5)
        # save the plot to a file
        filename = f"Trial\plot.png"
        plt.savefig(filename, dpi = 100, bbox_inches='tight')

        return filename

    def resize_flatten(self, path):
        img_array=imread(path)
        img_resized=resize(img_array,(150,150,3)).flatten()

        return img_resized
    
    def predcitIf(self, x_test):
        # load saved model 
        loaded_model = pickle.load(open('Models\\finalized_model_V1_NODIR.sav', 'rb'))
        # result = loaded_model.score(x_test, y_test)

        return loaded_model.predict(x_test)


In [ ]:
def main(): 
    count = 0 
    start = input(f"Type 'ST' to start running")
    x_array = np.array([])
    y_array = np.array([])
    while start == "ST":
        global tap_instance
        tap_instance = TapSDK()
        fun = Functionality()
        tap_instance.run()

        # tap_instance.register_connection_events(on_connect)
        # tap_instance.register_disconnection_events(on_disconnect)
        tap_instance.register_mouse_events(fun.on_mouse_event)
        tap_instance.set_input_mode(TapInputMode(modes[2]))   

        print("Recording Data...")
        trial = True 
        while trial:
            if(keyboard.is_pressed("q")):
                trial = False
                print("You have chosen to end this trial.")
            else: 
                if str(fun.current_x) != "None":
                    x_array = np.append(x_array, [fun.current_x])
                    y_array = np.append(y_array, [fun.current_y])
                pass
        
        if input("type 'y'") == 'y':
            x_array = np.unique(x_array)
            y_array = np.unique(x_array)
            
            x_array, y_array = fun.reltoNonrel(x_array, y_array)
            plot_path  = fun.plotfig(x_array, y_array)
            resized_image = fun.resize_flatten(plot_path)
            result = fun.predcitIf(resized_image.reshape(1, -1))
            print(result)
        else:
            start = "Trial Ended"
            print(start)

if __name__ == "__main__":
    main() 

### Import Libraries

In [ ]:
import lazypredict
from lazypredict.Supervised import LazyClassifier